In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
import torch.optim as optim


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)


Using device:  cpu


In [5]:
df = pd.read_csv("fashion-mnist.csv")
df.shape


(10000, 785)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42
)


X_test.shape


(2000, 784)

In [7]:
X_train = X_train / 255.0
X_test = X_test / 255.0


In [8]:
class MyCustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [9]:
train_dataset = MyCustomDataset(X_train, y_train)
test_dataset = MyCustomDataset(X_test, y_test)
len(train_dataset)
# len(test_dataset)


8000

In [13]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)


In [14]:
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        return self.model(x)


In [15]:
learning_rate = 0.1
epochs = 100


In [16]:
model = MyNN(X_train.shape[1])
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [17]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(
            device
        )
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        total_epoch_loss += loss.item()

    avg_epoch_loss = total_epoch_loss / len(train_loader)
    if epoch % 10 == 0:
        print(
            "Epoch: {}/{}... ".format(epoch, epochs),
            "Loss: {:.6f}...".format(avg_epoch_loss),
        )


Epoch: 0/100...  Loss: 1.409057...
Epoch: 10/100...  Loss: 0.406682...
Epoch: 20/100...  Loss: 0.301464...
Epoch: 30/100...  Loss: 0.230024...
Epoch: 40/100...  Loss: 0.168378...
Epoch: 50/100...  Loss: 0.139590...
Epoch: 60/100...  Loss: 0.110111...
Epoch: 70/100...  Loss: 0.082474...
Epoch: 80/100...  Loss: 0.076510...
Epoch: 90/100...  Loss: 0.039478...


In [18]:
model.eval()


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [19]:
total = 0
correct = 0

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(
            device
        )
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    print(
        "Accuracy of the network on the 1200 test images: %d %%"
        % (100 * correct / total)
    )


Accuracy of the network on the 1200 test images: 86 %
